In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import stan
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm
from scipy import stats
from scipy.stats import pearsonr
import pickle

FONT_SIZE = 8

plt.rc('font', size=FONT_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=FONT_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=FONT_SIZE)    # legend fontsize
plt.rc('figure', titlesize=FONT_SIZE)  # fontsize of the figure title
plt.rcParams['pdf.fonttype'] = 42

In [6]:
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt')
# plt.style.use('seaborn-white')

#%%

### load data ###
dat_RC = pd.read_csv('smFISH_RC.csv')

jtime_RC = dat_RC['Time'].values
AreaNormed_RC = dat_RC['AreaNormed'].values
CountsNr1d1_RC = dat_RC['Counts Nr1d1'].values
CountsCry1_RC = dat_RC['Counts Cry1'].values

dat_BC = pd.read_csv('smFISH_BC.csv')

jtime_BC = dat_BC['Time'].values
AreaNormed_BC = dat_BC['AreaNormed'].values
CountsBmal1_BC = dat_BC['Counts Bmal1'].values
CountsCry1_BC = dat_BC['Counts Cry1'].values

CountsCry1TOT = np.concatenate((CountsCry1_BC,CountsCry1_RC))
jtime_Cry1TOT = np.concatenate((jtime_BC,jtime_RC))
AreaCry1TOT = np.concatenate((AreaNormed_BC,AreaNormed_RC))

w = 2 * np.pi / 24 

TimevecRC = np.array([21.,25.,29.,33.,37.,41.]).reshape(-1,1)
TimevecBC = np.array([17.,21.,25.,29.,33.,37.,41.]).reshape(-1,1)

In [13]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
asyncio.run(asyncio.sleep(1))

In [30]:
#%%
# load Fourier parameters
Nr1d1_params, Cry1_params, Bmal1_params = pickle.load(open('FourierParams.pkl', 'rb'))

#%%

# MODEL 1

model1 = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC; 
    
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}
 

"""

#%%


dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [ ]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=4, num_samples=2000)

with open('model_1.pkl', 'wb') as f:
    pickle.dump(sm, f)
    
with open('fit_1.pkl', 'wb') as g:
    pickle.dump(fit, g)         

Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 6.9s
Building: 7.0s
Building: 

In file included from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/tbb/concurrent_unordered_map.h:26,
                 from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core/profiling.hpp:10,
                 from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core.hpp:53,
                 from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/kappa/.cache/httpstan/4.13.0/models/owkzch4l/model_owkzch4l.cpp:2:
/home/kappa/miniforge3/lib/python3.10/site-packages/httpstan/include/tbb/internal/_concurrent_unordered_impl.h: In instantiation of ‘void tbb::interface5::internal::concurrent_unordered_bas

Building: 15.6s
Building: 15.8s
Building: 15.9s
Building: 16.0s
Building: 16.1s
Building: 16.2s
Building: 16.3s
Building: 16.4s
Building: 16.5s
Building: 16.6s
Building: 16.7s
Building: 16.8s
Building: 16.9s
Building: 17.0s
Building: 17.1s
Building: 17.2s
Building: 17.3s
Building: 17.4s
Building: 17.5s
Building: 17.6s
Building: 17.7s
Building: 17.8s
Building: 18.0s
Building: 18.1s
Building: 18.2s
Building: 18.3s
Building: 18.4s
Building: 18.5s
Building: 18.6s
Building: 18.7s
Building: 18.8s
Building: 18.9s
Building: 19.0s
Building: 19.1s
Building: 19.2s
Building: 19.3s
Building: 19.4s
Building: 19.5s
Building: 19.6s
Building: 19.7s
Building: 19.8s
Building: 19.9s
Building: 20.0s
Building: 20.1s
Building: 20.3s
Building: 20.4s
Building: 20.5s
Building: 20.6s
Building: 20.7s
Building: 20.8s
Building: 20.9s
Building: 21.0s
Building: 21.1s
Building: 21.2s
Building: 21.3s
Building: 21.4s
Building: 21.5s
Building: 21.6s
Building: 21.7s
Building: 21.8s
Building: 21.9s
Building: 22.0s
Building

Building: 29.4s, done.
Messages from stanc:
Warning in '/tmp/httpstan_pwehya34/model_owkzch4l.stan', line 74, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pwehya34/model_owkzch4l.stan', line 73, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pwehya34/model_owkzch4l.stan', line 72, column 24: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pwehya34/model_owkzch4l.stan', line 71, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pwehya34/model_owkzch4l.stan', line 70, column

In [ ]:
#%%

# MODEL 2

model1 = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
    beta_v_Cry1~ normal(0,100);
    beta_v_Nr1d1~ normal(0,100);
    beta_v_Bmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}    

"""

In [ ]:
#%%

dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [ ]:
sm = pystan.StanModel(model_code=model1)

fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))

with open('model_2.pkl', 'wb') as f:
    pickle.dump(sm, f)
    
with open('fit_2.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [ ]:
#%%
# MODEL 3 - phase noise

model1 = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    real kappa;
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real phi_i_add_RC[N1];
    real phi_i_add_BC[N2];
    real<lower=0,upper=2> amp_scal;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    for (n in 1:N1) {
        phi_i_add_RC[n] ~ von_mises(0, kappa);
    }
    for (n in 1:N2) {
        phi_i_add_BC[n] ~ von_mises(0, kappa);
    }
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}        

"""

#%%

In [ ]:
sm = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit.extract(permuted=True)

# load parameters from model M2
freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)
burstCry1 = np.mean(a['burstCry1'],axis=0)
burstNr1d1 = np.mean(a['burstNr1d1'],axis=0)
burstBmal1 = np.mean(a['burstBmal1'],axis=0)

In [ ]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'kappa' : 2,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'burstCry1' : burstCry1,
    'burstNr1d1' : burstNr1d1,
    'burstBmal1' : burstBmal1,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}
#%%

In [ ]:
sm = pystan.StanModel(model_code=model1)

fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=8, control=dict(adapt_delta=0.95))

with open('model_3.pkl', 'wb') as f:
    pickle.dump(sm, f)
    
with open('fit_3.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [ ]:
#%%     

# MODEL4

model1 = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    matrix[N1,2] eta_RC;
    matrix[N2,2] eta_BC;
    real<lower=0> stdevCry1;
    real<lower=0> stdevNr1d1; 
    real<lower=0> stdevBmal1; 
    cholesky_factor_corr[2] L_RC;
    cholesky_factor_corr[2] L_BC;
}
transformed parameters{
    matrix[2, 2] corr_RC;
    matrix[2, 2] cov_RC;
    vector[2] mu_vec_RC;
    vector[2] sigma_vec_RC;
    matrix[2, 2] corr_BC;
    matrix[2, 2] cov_BC;
    vector[2] mu_vec_BC;
    vector[2] sigma_vec_BC;
    corr_RC = L_RC*L_RC';
    corr_BC = L_BC*L_BC';
    mu_vec_RC[1] = burstCry1;
    mu_vec_RC[2] = burstNr1d1;
    mu_vec_BC[1] = burstCry1;
    mu_vec_BC[2] = burstBmal1;
    sigma_vec_RC[1] = stdevCry1;
    sigma_vec_RC[2] = stdevNr1d1;
    sigma_vec_BC[1] = stdevCry1;
    sigma_vec_BC[2] = stdevBmal1; 
    cov_RC = quad_form_diag(corr_RC, sigma_vec_RC);
    cov_BC = quad_form_diag(corr_BC, sigma_vec_BC);
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    vector[2] rescaled_eta;
    for ( i in 1:N1 ) {
        rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
        b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
        b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
        b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
        b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    to_vector(eta_RC) ~ normal(0, 1);
    to_vector(eta_BC) ~ normal(0, 1);
    burstCry1 ~ normal(0, 100);
    burstNr1d1 ~ normal(0, 100);
    burstBmal1 ~ normal(0, 100);
    L_RC ~ lkj_corr_cholesky(4.0);
    L_BC ~ lkj_corr_cholesky(4.0);
    stdevCry1 ~ normal(0, 100);
    stdevNr1d1 ~ normal(0, 100);
    stdevBmal1 ~ normal(0, 100);
}
generated quantities{ 
vector[N1] log_lik1;
vector[N1] log_lik2;
vector[N2] log_lik3;
vector[N2] log_lik4;
real mu1;
real mu2;
real mu3;
real mu4;
real r1;
real r2;
real r3;
real r4;
real b1;
real b2;
real b3;
real b4;
real f1;
real f2;
real f3;
real f4;
vector[2] rescaled_eta;
for ( i in 1:N1 ) {
    rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
    b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
    b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
    f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
    f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
    mu1 = b1*f1;
    mu2 = b2*f2;
    r1 = f1;
    r2 = f2;
    log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
    log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
} 
for ( i in 1:N2 ) {
    rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
    b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
    b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
    f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
    f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
    mu3 = b3*f3;
    mu4 = b4*f4;
    r3 = f3;
    r4 = f4;
    log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
    log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
} 


}       

"""

In [ ]:
# load parameters from model M2

sm = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit.extract(permuted=True)    

freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)

In [ ]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [ ]:
sm = pystan.StanModel(model_code=model1)

fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))

with open('model_4.pkl', 'wb') as f:
    pickle.dump(sm, f)
    
with open('fit_4.pkl', 'wb') as g:
    pickle.dump(fit, g)   
    
a = fit.extract(permuted=True)